## Hand landmark visualization
This notebook shows an example how to visualize hand landmarks we generated via [Mediapipe](https://google.github.io/mediapipe/).

In [52]:
import cv2
import json
import pandas as pd
from matplotlib import pyplot as plt

In [44]:
sample_video = 'path/to/sample_video/44e8d2a0-7e01-450b-90b0-beb7400d2c1e.mp4'
sample_id = sample_video.split('/')[-1][:-4]

In [13]:
# get video width and height
df = pd.read_csv('path/to/annotations.csv', sep='\t')
target = df[df['attachment_id'] == sample_id]
width, height = int(target['width']), int(target['height'])

In [42]:
with open('path/to/slovo_mediapipe.json') as file:
    landmarks = json.load(file)

In [96]:
# calculate absolute landmark values
landmark_abs = landmarks[sample_id]
for frame in landmark_abs:
    for hand in frame:
        points = frame[hand]
        for point in points:
            point['x'] *= width
            point['y'] *= height

For each hand in each frame we have 21 keypoints

In [53]:
from IPython.display import Image
Image(url= "https://developers.google.com/static/mediapipe/images/solutions/hand-landmarks.png")

*Source: [MediaPipe Solutions Guide](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker)*

In [97]:
vidcap = cv2.VideoCapture(sample_video)
writer = cv2.VideoWriter("video_landmarks.mkv", cv2.VideoWriter_fourcc(*"h264"), 30.0, (width, height))
for landmark in landmark_abs:
    success, frame = vidcap.read()
    for hand in landmark:
        points = landmark[hand]
        for point in points:
             out = cv2.circle(frame, [int(point['x']), int(point['y'])], 1, [0,255,0], 3)
        writer.write(frame)

In [98]:
from IPython.display import display, HTML

video_path = "video_landmarks.mkv"
video_tag = f'<video controls autoplay src="{video_path}" width="640" height="1024"/>'
display(HTML(video_tag))